<a href="https://colab.research.google.com/github/betsenara/UCSD-ML-Engineering-AI/blob/main/Student_MLE_MiniProject_Fine_TuningSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project: Transfer Learning with Keras

Transfer learning is a machine learning technique where a model trained on one task is used as a starting point to solve a different but related task. Instead of training a model from scratch, transfer learning leverages the knowledge learned from the source task and applies it to the target task. This approach is especially useful when the target task has limited data or computational resources.

In transfer learning, the pre-trained model, also known as the "base model" or "source model," is typically trained on a large dataset and a more general problem (e.g., image classification on ImageNet, a vast dataset with millions of labeled images). The knowledge learned by the base model in the form of feature representations and weights captures common patterns and features in the data.

To perform transfer learning, the following steps are commonly followed:

1. Pre-training: The base model is trained on a source task using a large dataset, which can take a considerable amount of time and computational resources.

2. Feature Extraction: After pre-training, the base model is used as a feature extractor. The last few layers (classifier layers) of the model are discarded, and the remaining layers (feature extraction layers) are retained. These layers serve as feature extractors, producing meaningful representations of the data.

3. Fine-tuning: The feature extraction layers and sometimes some of the earlier layers are connected to a new set of layers, often called the "classifier layers" or "task-specific layers." These layers are randomly initialized, and the model is trained on the target task with a smaller dataset. The weights of the base model can be frozen during fine-tuning, or they can be allowed to be updated with a lower learning rate to fine-tune the model for the target task.

Transfer learning has several benefits:

1. Reduced training time and resource requirements: Since the base model has already learned generic features, transfer learning can save time and resources compared to training a model from scratch.

2. Improved generalization: Transfer learning helps the model generalize better to the target task, especially when the target dataset is small and dissimilar from the source dataset.

3. Better performance: By starting from a model that is already trained on a large dataset, transfer learning can lead to better performance on the target task, especially in scenarios with limited data.

4. Effective feature extraction: The feature extraction layers of the pre-trained model can serve as powerful feature extractors for different tasks, even when the task domains differ.

Transfer learning is commonly used in various domains, including computer vision, natural language processing (NLP), and speech recognition, where pre-trained models are fine-tuned for specific applications like object detection, sentiment analysis, or speech-to-text.

In this mini-project you will perform fine-tuning using Keras with a pre-trained VGG16 model on the CIFAR-10 dataset.

First, import all the libraries you'll need.

In [84]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [85]:
import tensorflow as tf

# Check for GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


The CIFAR-10 dataset is a widely used benchmark dataset in the field of computer vision and machine learning. It stands for the "Canadian Institute for Advanced Research 10" dataset. CIFAR-10 was created by researchers at the CIFAR institute and was originally introduced as part of the Neural Information Processing Systems (NIPS) 2009 competition.

The dataset consists of 60,000 color images, each of size 32x32 pixels, belonging to ten different classes. Each class contains 6,000 images. The ten classes in CIFAR-10 are:

1. Airplane
2. Automobile
3. Bird
4. Cat
5. Deer
6. Dog
7. Frog
8. Horse
9. Ship
10. Truck

The images are evenly distributed across the classes, making CIFAR-10 a balanced dataset. The dataset is divided into two sets: a training set and a test set. The training set contains 50,000 images, while the test set contains the remaining 10,000 images.

CIFAR-10 is often used for tasks such as image classification, object recognition, and transfer learning experiments. The relatively small size of the images and the variety of classes make it a challenging dataset for training machine learning models, especially deep neural networks. It also serves as a good dataset for teaching and learning purposes due to its manageable size and straightforward class labels.

Here are your tasks:

1. Load the CIFAR-10 dataset after referencing the documentation [here](https://keras.io/api/datasets/cifar10/).
2. Normalize the pixel values so they're all in the range [0, 1].
3. Apply One Hot Encoding to the train and test labels using the [to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) function.
4. Further split the the training data into training and validation sets using [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Use only 10% of the data for validation.  

In [86]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [87]:
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [88]:
# Normalize the pixel values to [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

In [89]:
# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [90]:
# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [16]:
#Since the input shape expected by VGG16 (224, 224, 3) we resize the images:
# Resizing the images in the data:
#x_train = tf.image.resize(x_train, (224, 224))
#x_test = tf.image.resize(x_test, (224, 224))
#x_val = tf.image.resize(x_val, (224, 224))


VGG16 (Visual Geometry Group 16) is a deep convolutional neural network architecture that was developed by the Visual Geometry Group at the University of Oxford. It was proposed by researchers Karen Simonyan and Andrew Zisserman in their paper titled "Very Deep Convolutional Networks for Large-Scale Image Recognition," which was presented at the International Conference on Learning Representations (ICLR) in 2015.

The VGG16 architecture gained significant popularity for its simplicity and effectiveness in image classification tasks. It was one of the pioneering models that demonstrated the power of deeper neural networks for visual recognition tasks.

Key characteristics of the VGG16 architecture:

1. Architecture: VGG16 consists of a total of 16 layers, hence the name "16." These layers are stacked one after another, forming a deep neural network.

2. Convolutional Layers: The main building blocks of VGG16 are the convolutional layers. It primarily uses 3x3 convolutional filters throughout the network, which allows it to capture local features effectively.

3. Max Pooling: After each set of convolutional layers, VGG16 applies max-pooling layers with 2x2 filters and stride 2, which halves the spatial dimensions (width and height) of the feature maps and reduces the number of parameters.

4. Fully Connected Layers: Towards the end of the network, VGG16 has fully connected layers that act as a classifier to make predictions based on the learned features.

5. Activation Function: The network uses the Rectified Linear Unit (ReLU) activation function for all hidden layers, which helps with faster convergence during training.

6. Number of Filters: The number of filters in each convolutional layer is relatively small compared to more recent architectures like ResNet or InceptionNet. However, stacking multiple layers allows VGG16 to learn complex hierarchical features.

7. Output Layer: The output layer consists of 1000 units, corresponding to 1000 ImageNet classes. VGG16 was originally trained on the large-scale ImageNet dataset, which contains millions of images from 1000 different classes.

VGG16 was instrumental in showing that increasing the depth of a neural network can significantly improve its performance on image recognition tasks. However, the main drawback of VGG16 is its high number of parameters, making it computationally expensive and memory-intensive to train. Despite this limitation, VGG16 remains an essential benchmark architecture and has paved the way for even deeper and more efficient models in the field of computer vision, such as ResNet, DenseNet, and EfficientNet.

Here are your tasks:

1. Load [VGG16](https://keras.io/api/applications/vgg/#vgg16-function) as a base model. Make sure to exclude the top layer.
2. Freeze all the layers in the base model. We'll be using these weights as a feature extraction layer to forward to layers that are trainable.

In [91]:
# Load the pre-trained VGG16 model (excluding the top classifier)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [92]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

Now, we'll add some trainable layers to the base model.

1. Using the base model, add a [GlobalAveragePooling2D](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) layer, followed by a [Dense](https://keras.io/api/layers/core_layers/dense/) layer of length 256 with ReLU activation. Finally, add a classification layer with 10 units, corresponding to the 10 CIFAR-10 classes, with softmax activation.
2. Create a Keras [Model](https://keras.io/api/models/model/) that takes in approproate inputs and outputs.

In [93]:
# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)

In [94]:
# Add a fully connected layer with 256 units and ReLU activation
x = Dense(256, activation='relu')(x)

In [95]:
# Add the final classification layer with 10 units (for CIFAR-10 classes) and softmax activation
predictions = Dense(10, activation='softmax')(x)

In [96]:
# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=predictions)

With your model complete it's time to train it and assess its performance.

1. Compile your model using an appropriate loss function. Feel free to play around with the optimizer, but a good starting optimizer might be Adam with a learning rate of 0.001.
2. Fit your model on the training data. Use the validation data to print the accuracy for each epoch. Try training for 10 epochs. Note, training can take a few hours so go ahead and grab a cup of coffee.

**Optional**: See if you can implement an [Early Stopping](https://keras.io/api/callbacks/early_stopping/) criteria as a callback function.

In [97]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [98]:
# Train the model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10)

Epoch 1/10
1407/1407 [==============================] - 14s 9ms/step - loss: 1.3567 - accuracy: 0.5281 - val_loss: 1.2412 - val_accuracy: 0.5556
Epoch 2/10
1407/1407 [==============================] - 13s 9ms/step - loss: 1.1811 - accuracy: 0.5878 - val_loss: 1.1839 - val_accuracy: 0.5758
Epoch 3/10
1407/1407 [==============================] - 13s 9ms/step - loss: 1.1165 - accuracy: 0.6102 - val_loss: 1.1444 - val_accuracy: 0.5962
Epoch 4/10
1407/1407 [==============================] - 13s 9ms/step - loss: 1.0622 - accuracy: 0.6284 - val_loss: 1.1276 - val_accuracy: 0.6074
Epoch 5/10
1407/1407 [==============================] - 13s 9ms/step - loss: 1.0195 - accuracy: 0.6426 - val_loss: 1.0943 - val_accuracy: 0.6158
Epoch 6/10
1407/1407 [==============================] - 13s 9ms/step - loss: 0.9799 - accuracy: 0.6553 - val_loss: 1.1224 - val_accuracy: 0.6098
Epoch 7/10
1407/1407 [==============================] - 13s 9ms/step - loss: 0.9430 - accuracy: 0.6688 - val_loss: 1.1181 - val_ac

Validation accuracy is 0.6140, training accuracy is 0.7021. It better than random guess. The model is underfitting but training accuracy is higher than validation accuracy. Now we will try fine tuning.




In [99]:
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size = 32,validation_data=(x_val, y_val), epochs=10)


Epoch 1/10
1407/1407 [==============================] - 46s 30ms/step - loss: 0.7534 - accuracy: 0.7353 - val_loss: 0.8023 - val_accuracy: 0.7234
Epoch 2/10
1407/1407 [==============================] - 42s 30ms/step - loss: 0.5522 - accuracy: 0.8048 - val_loss: 0.7267 - val_accuracy: 0.7542
Epoch 3/10
1407/1407 [==============================] - 42s 30ms/step - loss: 0.4298 - accuracy: 0.8462 - val_loss: 0.6835 - val_accuracy: 0.7754
Epoch 4/10
1407/1407 [==============================] - 42s 30ms/step - loss: 0.3338 - accuracy: 0.8824 - val_loss: 0.6884 - val_accuracy: 0.7792
Epoch 5/10
1407/1407 [==============================] - 42s 30ms/step - loss: 0.2559 - accuracy: 0.9102 - val_loss: 0.6876 - val_accuracy: 0.7844
Epoch 6/10
1407/1407 [==============================] - 42s 30ms/step - loss: 0.1952 - accuracy: 0.9328 - val_loss: 0.7442 - val_accuracy: 0.7858
Epoch 7/10
1407/1407 [==============================] - 42s 30ms/step - loss: 0.1471 - accuracy: 0.9502 - val_loss: 0.7284 -

Validation accuracy is 0.8030 and training accuracy is 0.9760, clearly overfitting. To balance is we will use data augmentation.

In [100]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator with augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,        # Rotate images by up to 20 degrees
    width_shift_range=0.2,    # Shift images horizontally by up to 20% of the width
    height_shift_range=0.2,   # Shift images vertically by up to 20% of the height
    shear_range=0.2,          # Apply shear transformation with up to 20 degrees
    zoom_range=0.2,           # Zoom in or out by up to 20%
    horizontal_flip=True,     # Flip images horizontally with a probability of 50%
    fill_mode='nearest'       # Fill in newly created pixels with the nearest pixel value
)

In [101]:
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=10,
    validation_data=(x_val, y_val),
)

Epoch 1/10
704/704 [==============================] - 28s 39ms/step - loss: 0.7749 - accuracy: 0.7339 - val_loss: 0.6391 - val_accuracy: 0.7894
Epoch 2/10
704/704 [==============================] - 27s 39ms/step - loss: 0.6436 - accuracy: 0.7738 - val_loss: 0.5658 - val_accuracy: 0.8122
Epoch 3/10
704/704 [==============================] - 27s 38ms/step - loss: 0.6025 - accuracy: 0.7868 - val_loss: 0.5466 - val_accuracy: 0.8134
Epoch 4/10
704/704 [==============================] - 27s 38ms/step - loss: 0.5769 - accuracy: 0.7984 - val_loss: 0.5728 - val_accuracy: 0.8128
Epoch 5/10
704/704 [==============================] - 27s 39ms/step - loss: 0.5500 - accuracy: 0.8063 - val_loss: 0.5698 - val_accuracy: 0.8086
Epoch 6/10
704/704 [==============================] - 27s 38ms/step - loss: 0.5267 - accuracy: 0.8146 - val_loss: 0.5451 - val_accuracy: 0.8210
Epoch 7/10
704/704 [==============================] - 27s 38ms/step - loss: 0.5106 - accuracy: 0.8181 - val_loss: 0.5173 - val_accuracy:

In [102]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 3s 8ms/step - loss: 0.5941 - accuracy: 0.8091
Test accuracy: 0.8090999722480774


Training accuracy is 0.8384 and test accuracy is 0.8091. The difference between them is decreased. Now we will keep fine tuning.

In [103]:
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=10,
    validation_data=(x_val, y_val),
)

Epoch 1/10
704/704 [==============================] - 27s 39ms/step - loss: 0.4446 - accuracy: 0.8423 - val_loss: 0.5001 - val_accuracy: 0.8346
Epoch 2/10
704/704 [==============================] - 27s 39ms/step - loss: 0.4316 - accuracy: 0.8473 - val_loss: 0.4687 - val_accuracy: 0.8436
Epoch 3/10
704/704 [==============================] - 27s 38ms/step - loss: 0.4159 - accuracy: 0.8538 - val_loss: 0.4951 - val_accuracy: 0.8390
Epoch 4/10
704/704 [==============================] - 27s 38ms/step - loss: 0.4001 - accuracy: 0.8580 - val_loss: 0.5016 - val_accuracy: 0.8352
Epoch 5/10
704/704 [==============================] - 27s 39ms/step - loss: 0.3946 - accuracy: 0.8602 - val_loss: 0.4994 - val_accuracy: 0.8372
Epoch 6/10
704/704 [==============================] - 27s 38ms/step - loss: 0.3828 - accuracy: 0.8646 - val_loss: 0.4775 - val_accuracy: 0.8440
Epoch 7/10
704/704 [==============================] - 27s 38ms/step - loss: 0.3708 - accuracy: 0.8694 - val_loss: 0.4965 - val_accuracy:

With your model trained, it's time to assess how well it performs on the test data.

1. Use your trained model to calculate the accuracy on the test set. Is the model performance better than random?
2. Experiment! See if you can tweak your model to improve performance.  

In [104]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 3s 8ms/step - loss: 0.5148 - accuracy: 0.8435
Test accuracy: 0.843500018119812
